# GUI trial

In [1]:
import sys
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QWidget, QPushButton, QVBoxLayout, 
    QLabel, QFileDialog, QHBoxLayout, QLineEdit, QComboBox, QMessageBox
)
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone

In [2]:
class LoanPredictionModel:
    
    def __init__(self, data_path=None, model=None):
        self.data_path = data_path
        self.data = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None
        self.model = model if model else RandomForestClassifier()
        self.trained_model = None
    
    def load_data(self):
        self.data = pd.read_csv(self.data_path)
    
    def split_data(self, target_column, test_size=0.2, random_state=42):
        X = self.data.drop(columns=[target_column])
        y = self.data[target_column]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    def run_cross_validation(self, cv=5, scoring='accuracy'):
        kf = KFold(n_splits=cv, shuffle=True, random_state=42)
        scores = cross_val_score(self.model, self.X_train, self.y_train, cv=kf, scoring=scoring)
        return scores.mean()
    
    def train_model(self):
        self.trained_model = clone(self.model)
        self.trained_model.fit(self.X_train, self.y_train)
    
    def test_model(self):
        y_pred = self.trained_model.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        macro_f1 = f1_score(self.y_test, y_pred, average='macro')
        weighted_f1 = f1_score(self.y_test, y_pred, average='weighted')
        return accuracy, macro_f1, weighted_f1
    
    def delete_model(self):
        self.trained_model = None
    
    def delete_data(self):
        self.data = None
        self.X_train, self.X_test, self.y_train, self.y_test = None, None, None, None

In [3]:
class LoanPredictionGUI(QWidget):
    
    def __init__(self):
        super().__init__()
        self.init_ui()
        self.model = LoanPredictionModel()
    
    def init_ui(self):
        self.setWindowTitle("Loan Prediction Model")
        self.setGeometry(100, 100, 400, 300)
        
        # Layout
        layout = QVBoxLayout()
        
        # Load Data
        self.load_button = QPushButton("Load Data")
        self.load_button.clicked.connect(self.load_data)
        layout.addWidget(self.load_button)
        
        # Split Data
        self.split_data_button = QPushButton("Split Data")
        self.split_data_button.clicked.connect(self.split_data)
        layout.addWidget(self.split_data_button)
        
        # Train Model
        self.train_button = QPushButton("Train Model")
        self.train_button.clicked.connect(self.train_model)
        layout.addWidget(self.train_button)
        
        # Cross Validation
        self.cv_button = QPushButton("Run Cross-Validation")
        self.cv_button.clicked.connect(self.run_cv)
        layout.addWidget(self.cv_button)
        
        # Test Model
        self.test_button = QPushButton("Test Model")
        self.test_button.clicked.connect(self.test_model)
        layout.addWidget(self.test_button)
        
        # Output Label
        self.output_label = QLabel("Output will be shown here.")
        layout.addWidget(self.output_label)
        
        # Delete Model
        self.delete_model_button = QPushButton("Delete Model")
        self.delete_model_button.clicked.connect(self.delete_model)
        layout.addWidget(self.delete_model_button)
        
        # Delete Data
        self.delete_data_button = QPushButton("Delete Data")
        self.delete_data_button.clicked.connect(self.delete_data)
        layout.addWidget(self.delete_data_button)
        
        self.setLayout(layout)
    
    def load_data(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Open CSV File", "", "CSV Files (*.csv)")
        if file_name:
            self.model.data_path = file_name
            self.model.load_data()
            self.output_label.setText("Data loaded successfully.")
    
    def split_data(self):
        if self.model.data is not None:
            target_column = "Loan_Status"  # Assuming 'Loan_Status' is the target column
            self.model.split_data(target_column)
            self.output_label.setText("Data split successfully.")
        else:
            QMessageBox.warning(self, "Error", "Load data first.")
    
    def train_model(self):
        if self.model.X_train is not None:
            self.model.train_model()
            self.output_label.setText("Model trained successfully.")
        else:
            QMessageBox.warning(self, "Error", "Split data first.")
    
    def run_cv(self):
        if self.model.X_train is not None:
            score = self.model.run_cross_validation()
            self.output_label.setText(f"Cross-validation accuracy: {score:.4f}")
        else:
            QMessageBox.warning(self, "Error", "Split data first.")
    
    def test_model(self):
        if self.model.trained_model is not None:
            accuracy, macro_f1, weighted_f1 = self.model.test_model()
            self.output_label.setText(f"Accuracy: {accuracy:.4f}, Macro F1: {macro_f1:.4f}, Weighted F1: {weighted_f1:.4f}")
        else:
            QMessageBox.warning(self, "Error", "Train the model first.")
    
    def delete_model(self):
        self.model.delete_model()
        self.output_label.setText("Model deleted.")
    
    def delete_data(self):
        self.model.delete_data()
        self.output_label.setText("Data deleted.")

In [4]:
app = QApplication(sys.argv)
window = LoanPredictionGUI()
window.show()
sys.exit(app.exec_())

2024-11-15 19:33:55.359 python[5984:250170] +[IMKClient subclass]: chose IMKClient_Legacy
2024-11-15 19:33:55.359 python[5984:250170] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
2024-11-15 19:33:59.304 python[5984:250170] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'


SystemExit: 0

/opt/homebrew/Caskroom/miniforge/base/envs/envp3t/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
